In [7]:
import tensorflow as tf
import time # for throughput measurements
import numpy as np

# enable logging
# import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
gpus = tf.config.list_physical_devices('GPU')
print('Number of GPUs available :', len(gpus))
if gpus:
    gpu_num = 0 # Number of the GPU to be used
    try:
        #tf.config.set_visible_devices([], 'GPU')
        tf.config.set_visible_devices(gpus[gpu_num], 'GPU')
        print('Only GPU number', gpu_num, 'used.')
        tf.config.experimental.set_memory_growth(gpus[gpu_num], True)
    except RuntimeError as e:
        print(e)


Number of GPUs available : 1
Only GPU number 0 used.


In [8]:
bs = 10000
n = 800
rank = 400
pcm = np.random.randint(2, size=(rank, n)) # a random binary matrix
pcm = tf.convert_to_tensor(pcm, dtype=tf.int32)
print(pcm)

tf.Tensor(
[[1 1 1 ... 1 0 0]
 [1 0 1 ... 1 0 0]
 [1 1 0 ... 0 1 1]
 ...
 [1 1 0 ... 1 1 0]
 [1 0 1 ... 1 1 0]
 [0 0 0 ... 1 0 1]], shape=(400, 800), dtype=int32)


In [9]:
class Masking_Model(tf.keras.Model):
# For storing error strings that the BP decoder failed to decode
    def __init__(self, pcm):
        super().__init__()        
        self.pcm = pcm
        self.rank, self.n = pcm.shape
      
    @tf.function(jit_compile=True, reduce_retracing=True) # XLA mode
#     @tf.function() # graph mode
    def call(self, bs, p):

        noise = tf.random.uniform([bs, self.n], 0, 1, tf.float32)
        noise = (noise < 0.5)
        e = tf.cast(noise, tf.int32) # random binary noise
        print(e)
        
        s = tf.matmul(self.pcm, tf.transpose(e, (1,0))) # shape [rank, bs], do not mod 2
        
        mask = ((tf.random.uniform([bs], 0, 1, tf.float32)) < 0.8) # random binary mask
        
        new_e = tf.boolean_mask(e, mask)
        new_s = tf.boolean_mask(s, mask, axis=1)
        
        inner_s = tf.matmul(self.pcm, tf.transpose(new_e, (1,0)))
        return tf.reduce_sum(tf.abs(new_s - inner_s))


In [11]:
model = Masking_Model(pcm)
result = model(bs, 0.1)
print(result)

2023-11-15 11:52:02.746283: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'bs' with dtype int32
	 [[{{node bs}}]]
2023-11-15 11:52:02.752318: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'bs' with dtype int32
	 [[{{node bs}}]]


Tensor("Cast:0", shape=(None, 800), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)


In [12]:
class Masking_Model_Buggy(tf.keras.Model):
# For storing error strings that the BP decoder failed to decode
    def __init__(self, pcm):
        super().__init__()        
        self.pcm = pcm
        self.rank, self.n = pcm.shape
      
    @tf.function(jit_compile=True, reduce_retracing=True) # XLA mode
#     @tf.function() # graph mode
    def call(self, bs, p):

        noise = tf.random.uniform([bs, self.n], 0, 1, tf.float32)
        noise = (noise < 0.5)
        e = tf.cast(noise, tf.int32) # random binary noise
        print(e)
        
        s = tf.matmul(self.pcm, tf.transpose(e, (1,0))) # shape [rank, bs], do not mod 2
        
        mask = ((tf.random.uniform([bs], 0, 1, tf.float32)) < 0.8) # random binary mask
        
        new_e = tf.boolean_mask(e, mask)
        new_s = tf.boolean_mask(s, mask, axis=1)

        return new_e, new_s

In [15]:
model_buggy = Masking_Model_Buggy(pcm)
e, s = model_buggy(bs, 0.1)
outer_s = tf.matmul(pcm, tf.transpose(e, (1,0)))
print(tf.reduce_sum(tf.abs(s - outer_s)))
print(tf.shape(e))
print(tf.shape(s))

2023-11-15 11:52:35.783221: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'bs' with dtype int32
	 [[{{node bs}}]]
2023-11-15 11:52:35.788081: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'bs' with dtype int32
	 [[{{node bs}}]]


Tensor("Cast:0", shape=(None, 800), dtype=int32)
tf.Tensor(45191284, shape=(), dtype=int32)
tf.Tensor([8040  800], shape=(2,), dtype=int32)
tf.Tensor([ 400 8040], shape=(2,), dtype=int32)
